In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
from IPython.display import Image
from scipy.stats import randint as randint
from scipy.stats import uniform
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split

try:
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import StratifiedKFold
except ImportError:
    from sklearn.cross_validation import GridSearchCV
    from sklearn.cross_validation import RandomizedSearchCV
    from sklearn.cross_validation import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline

C:\Users\Legat\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
df = pd.read_csv('trainl3.csv')

In [9]:
df.head(3)

,_id,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,target
0,df7489733b004bbe40d3d37b34f82419,54,technician,married,professional.course,unknown,no,no,cellular,aug,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,905a0b9a5456ee962223033473666be3,36,services,single,basic.4y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,02d69c07ec6d68e31d641edd45ce61cd,53,blue-collar,married,basic.4y,no,yes,no,cellular,aug,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,0


In [10]:
#Соседи
from dask.diagnostics import ProgressBar
y = df['target']
dfneib=df.drop(['target'], axis=1)
#dfneib=dfneib.drop(['contact'], axis=1)
#dfneib=dfneib.drop(['month'], axis=1)
#dfneib=dfneib.drop(['day_of_week'], axis=1)
#dfneib=dfneib.drop(['duration'], axis=1)
dfneib=dfneib.drop(['_id'], axis=1)
X=pd.get_dummies(dfneib)
X.shape
print("start1")
random_state = 42
metrics = ['roc_auc'] 
numNeighbors  = np.arange(1,150)
nFolds = 3
weight_options = ['distance','uniform']
algorithm = ['auto', 'ball_tree','kd_tree', 'brute']
#p = {1,2,0}
#weight_options = ["uniform", "distance"]
param_grid = dict(n_neighbors=numNeighbors,weights = weight_options,algorithm=algorithm)
cv = cross_validation.StratifiedKFold(y,nFolds)
nbmodel=KNeighborsClassifier()
#grid = GridSearchCV(nbmodel,param_grid=param_grid,scoring='roc_auc',cv=cv,verbose=0)
#grid.fit(X,y)
random_searchnieb = RandomizedSearchCV(nbmodel, param_distributions=param_grid, n_iter=200, 
                                   cv=cv, scoring='roc_auc', random_state=random_state, verbose=10)
with ProgressBar():
    random_searchnieb.fit(X, y)
model = random_searchnieb.best_params_
print(model)

start1
Fitting 3 folds for each of 200 candidates, totalling 600 fits
[CV] weights=distance, n_neighbors=14, algorithm=brute ...............


KeyboardInterrupt: 

In [14]:
from dask.diagnostics import ProgressBar
y = df['target']
dfneib=df.drop(['target'], axis=1)
#dfneib=dfneib.drop(['contact'], axis=1)
#dfneib=dfneib.drop(['month'], axis=1)
#dfneib=dfneib.drop(['day_of_week'], axis=1)
#dfneib=dfneib.drop(['duration'], axis=1)
dfneib=dfneib.drop(['_id'], axis=1)
X=pd.get_dummies(dfneib)
X.shape
nFolds = 3
tree = DecisionTreeClassifier()
max_depth  = np.arange(10,50)
max_features = np.arange(1,10)
min_samples_leaf = np.arange(1,5)
class_weight = ['balanced',None]
criterion = ['gini','entropy']
param_grid = dict(max_depth=max_depth,max_features = max_features,criterion=criterion,min_samples_leaf=min_samples_leaf,class_weight=class_weight)
cv = cross_validation.StratifiedKFold(y,nFolds)

random_searchtr = RandomizedSearchCV(tree, param_distributions=param_grid, n_iter=1500, 
                                   cv=cv, scoring='roc_auc', random_state=random_state, verbose=10)
with ProgressBar():
    random_searchtr.fit(X, y)
model = random_searchtr.best_params_
print(model)
print(random_searchtr.best_score_)

Fitting 3 folds for each of 1500 candidates, totalling 4500 fits
[CV] min_samples_leaf=3, max_features=7, max_depth=11, criterion=entropy, class_weight=None 
[CV]  min_samples_leaf=3, max_features=7, max_depth=11, criterion=entropy, class_weight=None, score=0.808156, total=   0.0s
[CV] min_samples_leaf=3, max_features=7, max_depth=11, criterion=entropy, class_weight=None 
[CV]  min_samples_leaf=3, max_features=7, max_depth=11, criterion=entropy, class_weight=None, score=0.834842, total=   0.0s
[CV] min_samples_leaf=3, max_features=7, max_depth=11, criterion=entropy, class_weight=None 
[CV]  min_samples_leaf=3, max_features=7, max_depth=11, criterion=entropy, class_weight=None, score=0.874702, total=   0.0s
[CV] min_samples_leaf=1, max_features=6, max_depth=30, criterion=entropy, class_weight=None 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


[CV]  min_samples_leaf=1, max_features=6, max_depth=30, criterion=entropy, class_weight=None, score=0.713898, total=   0.0s
[CV] min_samples_leaf=1, max_features=6, max_depth=30, criterion=entropy, class_weight=None 
[CV]  min_samples_leaf=1, max_features=6, max_depth=30, criterion=entropy, class_weight=None, score=0.700558, total=   0.0s
[CV] min_samples_leaf=1, max_features=6, max_depth=30, criterion=entropy, class_weight=None 
[CV]  min_samples_leaf=1, max_features=6, max_depth=30, criterion=entropy, class_weight=None, score=0.707882, total=   0.0s
[CV] min_samples_leaf=4, max_features=7, max_depth=30, criterion=entropy, class_weight=balanced 
[CV]  min_samples_leaf=4, max_features=7, max_depth=30, criterion=entropy, class_weight=balanced, score=0.841723, total=   0.0s
[CV] min_samples_leaf=4, max_features=7, max_depth=30, criterion=entropy, class_weight=balanced 


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.4s remaining:    0.0s


[CV]  min_samples_leaf=4, max_features=7, max_depth=30, criterion=entropy, class_weight=balanced, score=0.866019, total=   0.0s
[CV] min_samples_leaf=4, max_features=7, max_depth=30, criterion=entropy, class_weight=balanced 
[CV]  min_samples_leaf=4, max_features=7, max_depth=30, criterion=entropy, class_weight=balanced, score=0.847240, total=   0.0s
[CV] min_samples_leaf=2, max_features=7, max_depth=23, criterion=entropy, class_weight=None 
[CV]  min_samples_leaf=2, max_features=7, max_depth=23, criterion=entropy, class_weight=None, score=0.800716, total=   0.0s
[CV] min_samples_leaf=2, max_features=7, max_depth=23, criterion=entropy, class_weight=None 
[CV]  min_samples_leaf=2, max_features=7, max_depth=23, criterion=entropy, class_weight=None, score=0.804899, total=   0.0s
[CV] min_samples_leaf=2, max_features=7, max_depth=23, criterion=entropy, class_weight=None 


...

[CV]  min_samples_leaf=1, max_features=3, max_depth=25, criterion=gini, class_weight=balanced, score=0.703871, total=   0.0s
[CV] min_samples_leaf=1, max_features=3, max_depth=25, criterion=gini, class_weight=balanced 
[CV]  min_samples_leaf=1, max_features=3, max_depth=25, criterion=gini, class_weight=balanced, score=0.674860, total=   0.0s
[CV] min_samples_leaf=1, max_features=1, max_depth=34, criterion=gini, class_weight=balanced 
[CV]  min_samples_leaf=1, max_features=1, max_depth=34, criterion=gini, class_weight=balanced, score=0.663022, total=   0.0s
[CV] min_samples_leaf=1, max_features=1, max_depth=34, criterion=gini, class_weight=balanced 
[CV]  min_samples_leaf=1, max_features=1, max_depth=34, criterion=gini, class_weight=balanced, score=0.634115, total=   0.0s
[CV] min_samples_leaf=1, max_features=1, max_depth=34, criterion=gini, class_weight=balanced 
[CV]  min_samples_leaf=1, max_features=1, max_depth=34, criterion=gini, class_weight=balanced, score=0.630858, total=   0.0s

[CV]  min_samples_leaf=2, max_features=7, max_depth=19, criterion=entropy, class_weight=balanced, score=0.810383, total=   0.0s
[CV] min_samples_leaf=3, max_features=9, max_depth=46, criterion=gini, class_weight=None 
[CV]  min_samples_leaf=3, max_features=9, max_depth=46, criterion=gini, class_weight=None, score=0.836144, total=   0.0s
[CV] min_samples_leaf=3, max_features=9, max_depth=46, criterion=gini, class_weight=None 
[CV]  min_samples_leaf=3, max_features=9, max_depth=46, criterion=gini, class_weight=None, score=0.810304, total=   0.0s
[CV] min_samples_leaf=3, max_features=9, max_depth=46, criterion=gini, class_weight=None 
[CV]  min_samples_leaf=3, max_features=9, max_depth=46, criterion=gini, class_weight=None, score=0.835400, total=   0.0s
[CV] min_samples_leaf=1, max_features=9, max_depth=18, criterion=gini, class_weight=None 
[CV]  min_samples_leaf=1, max_features=9, max_depth=18, criterion=gini, class_weight=None, score=0.745590, total=   0.0s
[CV] min_samples_leaf=1, max

[CV]  min_samples_leaf=1, max_features=6, max_depth=45, criterion=entropy, class_weight=None, score=0.687006, total=   0.0s
[CV] min_samples_leaf=1, max_features=6, max_depth=45, criterion=entropy, class_weight=None 
[CV]  min_samples_leaf=1, max_features=6, max_depth=45, criterion=entropy, class_weight=None, score=0.689236, total=   0.0s
[CV] min_samples_leaf=2, max_features=8, max_depth=28, criterion=entropy, class_weight=None 
[CV]  min_samples_leaf=2, max_features=8, max_depth=28, criterion=entropy, class_weight=None, score=0.772425, total=   0.0s
[CV] min_samples_leaf=2, max_features=8, max_depth=28, criterion=entropy, class_weight=None 
[CV]  min_samples_leaf=2, max_features=8, max_depth=28, criterion=entropy, class_weight=None, score=0.798560, total=   0.0s
[CV] min_samples_leaf=2, max_features=8, max_depth=28, criterion=entropy, class_weight=None 
[CV]  min_samples_leaf=2, max_features=8, max_depth=28, criterion=entropy, class_weight=None, score=0.789260, total=   0.0s
[CV] min

[CV]  min_samples_leaf=4, max_features=5, max_depth=21, criterion=gini, class_weight=None, score=0.810440, total=   0.0s
[CV] min_samples_leaf=4, max_features=5, max_depth=21, criterion=gini, class_weight=None 
[CV]  min_samples_leaf=4, max_features=5, max_depth=21, criterion=gini, class_weight=None, score=0.826578, total=   0.0s
[CV] min_samples_leaf=4, max_features=5, max_depth=21, criterion=gini, class_weight=None 
[CV]  min_samples_leaf=4, max_features=5, max_depth=21, criterion=gini, class_weight=None, score=0.832809, total=   0.0s
[CV] min_samples_leaf=2, max_features=6, max_depth=17, criterion=gini, class_weight=None 
[CV]  min_samples_leaf=2, max_features=6, max_depth=17, criterion=gini, class_weight=None, score=0.837079, total=   0.0s
[CV] min_samples_leaf=2, max_features=6, max_depth=17, criterion=gini, class_weight=None 
[CV]  min_samples_leaf=2, max_features=6, max_depth=17, criterion=gini, class_weight=None, score=0.813625, total=   0.0s
[CV] min_samples_leaf=2, max_featur

[Parallel(n_jobs=1)]: Done 4500 out of 4500 | elapsed:  4.6min finished


In [7]:
from sklearn.linear_model import LogisticRegression
from dask.diagnostics import ProgressBar
max_iter =np.arange(1,100)
y = df['target']
dflog=df.drop(['target'], axis=1)

#dfneib=dfneib.drop(['contact'], axis=1)
#dfneib=dfneib.drop(['month'], axis=1)
#dfneib=dfneib.drop(['day_of_week'], axis=1)
#dfneib=dfneib.drop(['duration'], axis=1)
dflog=dflog.drop(['_id'], axis=1)
X=pd.get_dummies(dflog)
X.shape
nFolds = 3
C=np.arange(1,100)
cv = cross_validation.StratifiedKFold(y,nFolds)
solver_values = ['newton-cg','lbfgs', 'liblinear', 'sag'] #, 'lbfgs', 'liblinear', 'sag', 'saga'
#max_iter_values = [200]; #[50, 100, 150, 200, 250]
params = dict(solver=solver_values,C=C,max_iter=max_iter) #solver=solver_values, tol=[0.000001]
model=LogisticRegression(random_state=42)

searcher = RandomizedSearchCV(model, 
                              param_distributions=params, 
                              n_iter=150,
                              cv=cv, 
                              scoring='roc_auc', 
                              random_state=42, 
                              verbose=10)

with ProgressBar():
    searcher.fit(X, y)
model = searcher.best_params_
print(model)
print(searcher.best_score_)

Fitting 3 folds for each of 150 candidates, totalling 450 fits
[CV] solver=sag, max_iter=88, C=40 ...................................


C:\Users\Legat\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s


[CV] .... solver=sag, max_iter=88, C=40, score=0.874554, total=   1.5s
[CV] solver=sag, max_iter=88, C=40 ...................................


C:\Users\Legat\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.2s remaining:    0.0s


[CV] .... solver=sag, max_iter=88, C=40, score=0.877276, total=   1.6s
[CV] solver=sag, max_iter=88, C=40 ...................................


...

C:\Users\Legat\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


[CV]  solver=newton-cg, max_iter=73, C=81, score=0.932579, total=   1.5s
[CV] solver=liblinear, max_iter=23, C=83 .............................
[CV]  solver=liblinear, max_iter=23, C=83, score=0.926019, total=   0.1s
[CV] solver=liblinear, max_iter=23, C=83 .............................
[CV]  solver=liblinear, max_iter=23, C=83, score=0.933503, total=   0.1s
[CV] solver=liblinear, max_iter=23, C=83 .............................
[CV]  solver=liblinear, max_iter=23, C=83, score=0.930021, total=   0.1s
[CV] solver=lbfgs, max_iter=21, C=35 .................................
[CV] .. solver=lbfgs, max_iter=21, C=35, score=0.863840, total=   0.0s
[CV] solver=lbfgs, max_iter=21, C=35 .................................
[CV] .. solver=lbfgs, max_iter=21, C=35, score=0.863153, total=   0.0s
[CV] solver=lbfgs, max_iter=21, C=35 .................................
[CV] .. solver=lbfgs, max_iter=21, C=35, score=0.859067, total=   0.0s


[Parallel(n_jobs=1)]: Done 450 out of 450 | elapsed:  4.9min finished
C:\Users\Legat\AppData\Roaming\Python\Python36\site-packages\scipy\optimize\linesearch.py:461: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Legat\AppData\Roaming\Python\Python36\site-packages\scipy\optimize\linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


{'solver': 'newton-cg', 'max_iter': 96, 'C': 99}
0.932896608751


C:\Users\Legat\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [18]:
from sklearn.linear_model import LogisticRegression
df = pd.read_csv('trainl3.csv')
y = df['target']
dflog=df.drop(['target'], axis=1)
dflog=dflog.drop(['_id'], axis=1)
X=pd.get_dummies(dflog)

model=LogisticRegression(random_state=42,solver='newton-cg',max_iter=96,C=99)
model.fit(X,y)
dftest = pd.read_csv('testl3.csv')
dftest.head(2)
_id = dftest['_id']
dftestcheck=dftest.drop(['_id'], axis=1)
Xtest=pd.get_dummies(dftestcheck)
predictor= model
target = predictor.predict(Xtest)
res = pd.Series(target)
#df1['e'] = Series(np.random.randn(sLength), index=df1.index)
#result[]=_id.add(res)
dftest['target']=res
dfres=dftest[['_id','target']]

dfres.to_csv('D:\MyML\\lesson3\\resultles3.csv',sep=',', index=False)

C:\Users\Legat\AppData\Roaming\Python\Python36\site-packages\scipy\optimize\linesearch.py:461: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Legat\AppData\Roaming\Python\Python36\site-packages\scipy\optimize\linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Legat\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
